# CWRU Experiments
## Imports

In [1]:
import pandas as pd
import numpy as np

import lib.transformers as tf
from sklearn.pipeline import Pipeline

from lib.models import Ganomaly1d, Ganomaly2d, GanomalyFE, GanomalyNet
from lib.visualization import GANomalyBoard, rename_tensorboard_key

from skorch.callbacks import PassthroughScoring
import torch

from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split

from keras.datasets import mnist
import tensorflow

from sklearn.preprocessing import StandardScaler

In [2]:
cwruData0 = pd.read_parquet('data/cwru0.parquet')
cwruData1 = pd.read_parquet('data/cwru1.parquet')

cwruData = pd.concat([cwruData0, cwruData1])

## Settings

In [3]:
torch.manual_seed(0)

## Initializing Model(s)

In [5]:
def build_model(model, isize, max_epochs, directory, needs_feature_engineering = False, ngpu = 0, nz = 100, ndf = 64, ngf = 64, nc = 1, batch_size = 32, lr = 0.0002, beta1 = 0.5, beta2 = 0.999, workers = 2):
    
    if not needs_feature_engineering:
        output_model = GanomalyNet(
            model,
            module__isize = isize,
            module__nz=nz,
            module__ndf=ndf,
            module__ngf=ngf,
            module__nc=nc,
            module__ngpu=ngpu,

            device = torch.device("cuda:0") if torch.cuda.is_available() else 'cpu',

            criterion=torch.nn.BCELoss,

            optimizer_gen=torch.optim.Adam,
            optimizer_gen__lr=lr,
            optimizer_gen__betas=(beta1, beta2),

            optimizer_dis=torch.optim.Adam,
            optimizer_dis__lr=lr,
            optimizer_dis__betas=(beta1, beta2),

            batch_size=batch_size,
            max_epochs=max_epochs,

            train_split=False,  # not implemented
            iterator_train__shuffle=True,
            iterator_train__num_workers=workers,
            iterator_valid__num_workers=workers,

            callbacks=[
                PassthroughScoring('loss_dis', on_train=True),
                PassthroughScoring('loss_gen', on_train=True),
                PassthroughScoring('loss_gen_fra', on_train=True),
                PassthroughScoring('loss_gen_app', on_train=True),
                PassthroughScoring('loss_gen_lat', on_train=True),
                GANomalyBoard(SummaryWriter(log_dir= 'runs/' + directory), key_mapper = rename_tensorboard_key, close_after_train = False)
            ]
        )
    else:
            output_model = GanomalyNet(
            model,
            module__isize = isize,
            module__ngpu=ngpu,

            device = torch.device("cuda:0") if torch.cuda.is_available() else 'cpu',

            criterion=torch.nn.BCELoss,

            optimizer_gen=torch.optim.Adam,
            optimizer_gen__lr=lr,
            optimizer_gen__betas=(beta1, beta2),

            optimizer_dis=torch.optim.Adam,
            optimizer_dis__lr=lr,
            optimizer_dis__betas=(beta1, beta2),

            batch_size=batch_size,
            max_epochs=max_epochs,

            train_split=False,  # not implemented
            iterator_train__shuffle=True,
            iterator_train__num_workers=workers,
            iterator_valid__num_workers=workers,

            callbacks=[
                PassthroughScoring('loss_dis', on_train=True),
                PassthroughScoring('loss_gen', on_train=True),
                PassthroughScoring('loss_gen_fra', on_train=True),
                PassthroughScoring('loss_gen_app', on_train=True),
                PassthroughScoring('loss_gen_lat', on_train=True),
                GANomalyBoard(SummaryWriter(log_dir= 'runs/' + directory), key_mapper = rename_tensorboard_key, close_after_train = False)
            ]
        )
    
    return output_model

## Data Preparation
Building a pipeline of custom transformers to fetch and preprocess CWRU data.

In [6]:
common_preprocessing = \
Pipeline(steps=[
                ('DataSelector', tf.DataSelector(columns = ['fanEndData', 'driveEndData'], column_values = {'condition': ['Normal Baseline'], 'sampleRate': [12000]})),
                ('ArrayFlattener', tf.ArrayFlattener()),
                ('ArrayChunker', tf.ArrayChunker(3136)),
                ('ArrayFlattener2', tf.ArrayFlattener())
               ])

In [7]:
train, test = train_test_split(common_preprocessing.transform(cwruData), train_size = 400, test_size= 400, random_state = 0)

C:\Users\jlsachse\Documents\Bachelorarbeit\Implementation\ganomaly\lib\transformers.py:90: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X_ = np.array(X_)


In [15]:
max_epochs = 2

ganomaly_timeseries = build_model(Ganomaly1d, 3136, max_epochs, 'timeseries')
ganomaly_stacked_ts = build_model(Ganomaly2d, 56, max_epochs, 'stacked_timeseries')
ganomaly_fft = build_model(Ganomaly1d, 1568, max_epochs, 'fourier_transform')
ganomaly_stft = build_model(Ganomaly2d, 56, max_epochs, 'short_term_fouriers')
ganomaly_fe = build_model(GanomalyFE, 4, max_epochs, 'feature_extraction', True)

model_timeseries = \
Pipeline(steps=[
                ('StandardScaler', StandardScaler()),
                ('ArrayReshaper', tf.ArrayReshaper((1, 3136))),
                ('Model', ganomaly_timeseries)
               ])

model_stacked_ts = \
Pipeline(steps=[
                ('StandardScaler', StandardScaler()),
                ('ArrayReshaper', tf.ArrayReshaper((1, 56, 56))),
                ('Model', ganomaly_stacked_ts)
               ])

model_fe = \
Pipeline(steps=[
                ('FeatureExtractor', tf.FeatureExtractor(axis = 1)),
                ('StandardScaler', StandardScaler()),
                ('ArrayReshaper', tf.ArrayReshaper((1, 4, 4))),
                ('Model', ganomaly_fe)
               ])

model_stft = \
Pipeline(steps=[
                ('ArraySTFT', tf.ArraySTFT()),
                ('ArrayReshaper', tf.ArrayReshaper((3136))),
                ('StandardScaler', StandardScaler()),
                ('ArrayReshaper2', tf.ArrayReshaper((1, 56, 56))),
                ('Model', ganomaly_stft)
               ])

model_fft = \
Pipeline(steps=[
                ('ArrayFFT', tf.ArrayFFT()),
                ('StandardScaler', StandardScaler()),
                ('ArrayReshaper2', tf.ArrayReshaper((1, 1568))),
                ('Model', ganomaly_fft)
               ])


model_timeseries.fit(train)
model_stacked_ts.fit(train)
model_fe.fit(train)
model_stft.fit(train)
model_fft.fit(train)

  epoch    loss_dis    loss_gen    loss_gen_app    loss_gen_fra    loss_gen_lat    train_loss      dur
-------  ----------  ----------  --------------  --------------  --------------  ------------  -------
      1      1.0411      4.7213          0.8314          0.8944          2.9954        5.7624  32.2493
      2      1.0144      3.4045          0.8322          0.6845          1.8879        4.4189  32.5450
  epoch    loss_dis    loss_gen    loss_gen_app    loss_gen_fra    loss_gen_lat    train_loss      dur
-------  ----------  ----------  --------------  --------------  --------------  ------------  -------
      1      1.0075      4.3600          0.8300          0.5880          2.9420        5.3675  26.2801
      2      0.7574      2.5714          0.8326          0.0299          1.7089        3.3289  25.4524
  epoch    loss_dis    loss_gen    loss_gen_app    loss_gen_fra    loss_gen_lat    train_loss     dur
-------  ----------  ----------  --------------  --------------  ---------

Pipeline(memory=None,
         steps=[('ArrayFFT', ArrayFFT()),
                ('StandardScaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('ArrayReshaper2', ArrayReshaper(shape=(1, 1568))),
                ('Model',
                 <class 'lib.models.GanomalyNet'>[initialized](
  module_=Ganomaly1d(
    (l_fra): BCELoss()
    (l_app): L1Loss()
    (l_dis): L1Loss()
    (discriminator): DiscriminatorNet1d(
      (features): Sequent...
          (pyramid-128-batchnorm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (pyramid-128-256-convt): Conv1d(128, 256, kernel_size=(16,), stride=(4,), bias=False)
          (pyramid-relu-256): LeakyReLU(negative_slope=0.2, inplace=True)
          (pyramid-256-batchnorm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (final-256-1-convt): Conv1d(256, 100, kernel_size=(9,), stride=(1,), bias=False)
        )
     

In [9]:
import time


In [10]:
inputx = torch.zeros((64, 16, 1568))
print(inputx.shape)

l0 = torch.nn.Conv1d(16, 32, 16, 8, padding = 3, bias = False)

l1 = torch.nn.Conv1d(32, 64, 16, 4, padding = 6, bias = False)

l2 = torch.nn.Conv1d(64, 128, 16, 4, padding = 0, bias = False)

l3 = torch.nn.Conv1d(128, 256, 9, 4, padding = 0, bias = False)





lr2 = torch.nn.ConvTranspose1d(256, 128, 9, 4, padding = 0, bias = False)

lr3 = torch.nn.ConvTranspose1d(128, 64, 16, 4, padding = 0, output_padding=1, bias = False)

lr4 = torch.nn.ConvTranspose1d(64, 32, 16, 4, padding = 6, bias = False)

lr5 = torch.nn.ConvTranspose1d(32, 16, 16, 8, padding = 4, bias = False)


o0 = l0(inputx)
o1 = l1(o0)
o2 = l2(o1)
o3 = l3(o2)
print(o0.shape)
print(o1.shape)
print(o2.shape)
print(o3.shape)


or2 = lr2(o3)
or3 = lr3(or2)
or4 = lr4(or3)
or5 = lr5(or4)


#print(or1.shape)
print(or2.shape)
print(or3.shape)
print(or4.shape)
print(or5.shape)

torch.Size([64, 16, 1568])
torch.Size([64, 32, 195])
torch.Size([64, 64, 48])
torch.Size([64, 128, 9])
torch.Size([64, 256, 1])
torch.Size([64, 128, 9])
torch.Size([64, 64, 49])
torch.Size([64, 32, 196])
torch.Size([64, 16, 1568])


In [11]:
torch.Size([64, 16, 1568])
torch.Size([64, 32, 195])
torch.Size([64, 64, 48])
torch.Size([64, 128, 9])
torch.Size([64, 256, 1])
torch.Size([64, 128, 9])
torch.Size([64, 64, 49])
torch.Size([64, 32, 196])
torch.Size([64, 16, 1568])

torch.Size([64, 16, 1568])

In [12]:
torch.Size([64, 64, 196])
torch.Size([64, 128, 49])
torch.Size([64, 256, 9])
torch.Size([64, 512, 1])
torch.Size([64, 256, 9])
torch.Size([64, 128, 49])
torch.Size([64, 64, 196])
torch.Size([64, 32, 784])

torch.Size([64, 32, 784])

In [13]:
inputx = torch.zeros((64, 16, 55, 55))

l0 = torch.nn.Conv2d(16, 32, 4, 2, padding = 1, bias = False)

l1 = torch.nn.Conv2d(32, 64, 4, 2, padding = 1, bias = False)

l2 = torch.nn.Conv2d(64, 128, 4, 2, padding = 1, bias = False)

l3 = torch.nn.Conv2d(128, 256, 4, 2, padding = 1, bias = False)

l4 = torch.nn.Conv2d(256, 512, 3, 1, padding = 0, bias = False)

o0 = l0(inputx)
o1 = l1(o0)
o2 = l2(o1)
o3 = l3(o2)
o4 = l4(o3)

print(o0.shape)
print(o1.shape)
print(o2.shape)
print(o3.shape)
print(o4.shape)

lr1 = torch.nn.ConvTranspose2d(512, 256, 3, 1, padding = 0, bias = False)

lr2 = torch.nn.ConvTranspose2d(256, 128, 4, 2, padding = 1, output_padding = 1, bias = False)

lr3 = torch.nn.ConvTranspose2d(128, 64, 4, 2, padding = 1, bias = False)

lr4 = torch.nn.ConvTranspose2d(64, 32, 4, 2, padding = 1, bias = False)

lr5 = torch.nn.ConvTranspose2d(32, 16, 4, 2, padding = 1, bias = False)

or1 = lr1(o4)
or2 = lr2(or1)
or3 = lr3(or2)
or4 = lr4(or3)
or5 = lr5(or4)


print(or1.shape)
print(or2.shape)
print(or3.shape)
print(or4.shape)
print(or5.shape)

torch.Size([64, 32, 27, 27])
torch.Size([64, 64, 13, 13])
torch.Size([64, 128, 6, 6])
torch.Size([64, 256, 3, 3])
torch.Size([64, 512, 1, 1])
torch.Size([64, 256, 3, 3])
torch.Size([64, 128, 7, 7])
torch.Size([64, 64, 14, 14])
torch.Size([64, 32, 28, 28])
torch.Size([64, 16, 56, 56])


In [14]:
main = nn.Sequential()

nz = 100
cngf=

main.add_module('initial-{0}-{1}-convt'.format(nz, cngf),
                nn.ConvTranspose1d(nz, cngf, input_size, stride=1, output_padding = output_paddings[0] % 2, bias = False))
main.add_module('initial-{0}-relu'.format(cngf),
                nn.ReLU(True))
main.add_module('pyramid-{0}-batchnorm'.format(cngf),
                nn.BatchNorm1d(cngf))

for output_padding in output_paddings[1:-1]:
    main.add_module('pyramid-{0}-{1}-convt'.format(cngf, cngf // 2),
                    nn.ConvTranspose1d(cngf, cngf // 2, 16, stride=4, output_padding = output_padding, bias = False))
    main.add_module('pyramid-{0}-relu'.format(cngf // 2),
                    nn.ReLU(True))
    main.add_module('pyramid-{0}-batchnorm'.format(cngf // 2),
                    nn.BatchNorm1d(cngf // 2))

    cngf = cngf // 2


main.add_module('final-{0}-{1}-convt'.format(cngf, nc),
                    nn.ConvTranspose1d(cngf, nc, 16, stride=4, output_padding = output_paddings[-1], bias = False))
main.add_module('final-{0}-tanh'.format(nc),
                    nn.Tanh())

SyntaxError: invalid syntax (<ipython-input-14-b07dbda9e9df>, line 4)

In [ ]:
input_size = 1568


while input_size >= 46:
    input_size = input_size // 4
    
input_size

In [ ]:
inputx = torch.zeros((64, 128, 49, 4))

x = torch.nn.Conv2d(128, 64, 4, stride=1, bias = False)
y = torch.nn.ConvTranspose2d(64, 128, 4, stride=1, bias = False)
print(x(inputx).shape)
y(x(inputx)).shape

In [ ]:
input_size = 3136

output_paddings = []

n_layers = 0

while input_size >= 46:
    n_layers += 1
    output_paddings.append(input_size % 4)
    print(input_size)
    print(input_size % 4)
    input_size = input_size // 4

input_size // 4

In [ ]:
1568 // 4 // 4 // 4

In [ ]:
1568 / 39

In [ ]:
36 * 40 * 40

In [ ]:
1568 / 4 / 4 / 4

In [ ]:
3136 / 4 / 4 / 4 

In [ ]:
[len(array) for array in [[1], [2,3]]]

In [ ]:
stft(common_preprocessing.transform(cwruData)[:1], nperseg=112)

Train-test-split of the normal CWRU data.

In [ ]:
3136 // 4 // 4 // 4 // 4 

## GridSearchCV

In [ ]:
search_parameters = {
    'module_w_fra': list(range(0, 101, 10)),
    'module_w_app': list(range(0, 101, 10)),
    'module_w_lat': list(range(0, 101, 10)),
}

In [ ]:
#ganomaly_gs = GridSearchCV(ganomaly, search_parameters, refit=False, cv=4)

In [ ]:
#ganomaly_gs.fit(train)

## Training Visualization (best parameters)
Adding a TensorBoard for the visualization of the training.

In [ ]:
ganomaly_board = GANomalyBoard(SummaryWriter(), key_mapper = rename_tensorboard_key, close_after_train = False)
ganomaly_fe.callbacks += [ganomaly_board]

ganomaly_fe.callbacks

In [ ]:
ganomaly_fe.fit(train)

In [ ]:
ganomaly_fe.predict(test).mean()

In [ ]:
ganomaly_fe.predict(selection_pipeline.set_params(DataSelector__column_values = {'condition': ['Outer Race Fault'], 'sampleRate': [12000]}).transform(cwruData)[:400]).mean()

In [ ]:
56 * 56

In [ ]:
np.sqrt(1568)

In [ ]:
3136 // 4

In [ ]:
3136 // 2

In [ ]:
1568 / 4 / 4 / 4 